In [0]:
import tensorflow as tf
# import cv2 as cv
import os
import numpy as np
import glob
# # from random import shuffle
from google.colab import drive
import matplotlib.pyplot as plt
import csv
import re
import json
import math
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# from keras.layers import Embedding,Input
# from keras.models import Model
# from keras.optimizers import Adam
# from keras.layers import Bidirectional,LSTM,GlobalMaxPool1D,Dense
# from tensorflow.contrib import rnn
from sklearn.utils import shuffle

import tensorflow_hub as hub

In [2]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# !unzip -P '2Af0+z$;' '/content/drive/My Drive/audio_LLDs_eGeMAPS.zip' -d '/content/drive/My Drive/egemaps/'

In [0]:
# # train_data,train_lbl=[],[]
# target=[]
# m=[]
# f_data=[]
# EMB_DIM=3000
# pth=["/content/drive/My Drive/audio_functionnals_eGeMAPS/audio_functionnals_eGeMAPS/*.csv"]
# for filename in glob.glob(pth[0]):#CHANGE PATH
#   print(filename)
#   tmp=[]
#   target.append(filename.split("/")[-1].split(".")[0][:-11])
#   with open(filename,"rt") as f:
#       l_data=csv.reader(f)
#       for r in l_data:
# #           print(r)
#         x=r[0].split(";")
#         x.pop(0)
#         xx=list(map(np.float32,x))
# #         print(len(xx))
        
# #           xx=np.asarray(xx)
#         tmp.append(xx)
#   m.append(len(tmp))
#   mf=np.asarray(tmp) 
#   a = np.zeros((EMB_DIM-mf.shape[0],89))
#   mf=np.vstack((a,mf))
#   f_data.append(mf)
#   print(mf.shape)

In [0]:
def send(t_data,e_data):
    temp=[]
    ret_data=[]
    for ind,r in enumerate(e_data):
#         if(ind>0):
        x=r[0].split(";")
        a=x.pop(0)
        b=x.pop(0)
#         print(a,b)
#         print(x)
        xx=list(map(np.float32,x))
#         print(len(xx))
        temp.append(xx)
#     print(len(temp))
#     m.append(len(temp))
    for ind , r  in enumerate(t_data):
#         print(type(r[0]),r[0],r)
        if(ind>0):
            start_time,stop_time=math.floor(float(r[0])),math.floor(float(r[1]))
    #             print(r[0],r[1])
            if(stop_time<len(temp) and (stop_time-start_time)<100):
                for i in range(int(start_time),int(stop_time)):
#                     print(i)
                    ret_data.append(temp[i])
    m.append(len(ret_data))
    return ret_data

In [6]:
target=[]
m=[]
f_data=[]
emb_dim=1410
pth=["/content/drive/My Drive/audio_functionnals_eGeMAPS/audio_functionnals_eGeMAPS/*.csv"]
for filename in glob.glob(pth[0]):#CHANGE PATH
#   print(filename)
    tmp=[]
    if 'test' not in filename:
        print(filename)
        target.append(filename.split("/")[-1].split(".")[0][:-11])
    #     print(target)
        with open(filename,"rt") as f:
            b_data=csv.reader(f)
            for filename1 in glob.glob("/content/drive/My Drive/AVEC_trans/*"+target[-1]+"*.csv"):
                print(filename1)
                with open(filename1,"rt") as f1:
                    t_data=csv.reader(f1)
                    tmp=send(t_data,b_data)
#                 ege_data.append(tmp)
                    mf=np.asarray(tmp)
                    print(mf.shape)
                    a = np.zeros((emb_dim-mf.shape[0],88))
                    mf=np.vstack((a,mf))
#                 m.append(mf.shape[0])
                    f_data.append(mf)
        print(mf.shape)  

/content/drive/My Drive/audio_functionnals_eGeMAPS/audio_functionnals_eGeMAPS/development_01_OpenSMILE2.3.0_egemaps.csv
/content/drive/My Drive/AVEC_trans/development_01_Transcript.csv
(123, 88)
(1410, 88)
/content/drive/My Drive/audio_functionnals_eGeMAPS/audio_functionnals_eGeMAPS/development_14_OpenSMILE2.3.0_egemaps.csv
/content/drive/My Drive/AVEC_trans/development_14_Transcript.csv
(725, 88)
(1410, 88)
/content/drive/My Drive/audio_functionnals_eGeMAPS/audio_functionnals_eGeMAPS/development_02_OpenSMILE2.3.0_egemaps.csv
/content/drive/My Drive/AVEC_trans/development_02_Transcript.csv
(504, 88)
(1410, 88)
/content/drive/My Drive/audio_functionnals_eGeMAPS/audio_functionnals_eGeMAPS/development_03_OpenSMILE2.3.0_egemaps.csv
/content/drive/My Drive/AVEC_trans/development_03_Transcript.csv
(618, 88)
(1410, 88)
/content/drive/My Drive/audio_functionnals_eGeMAPS/audio_functionnals_eGeMAPS/development_04_OpenSMILE2.3.0_egemaps.csv
/content/drive/My Drive/AVEC_trans/development_04_Transc

In [7]:
print(max(m))

1407


In [8]:
print(len(f_data))
print(len(target))
f_data=np.asarray(f_data)
print(f_data.shape)

219
219
(219, 1410, 88)


In [0]:
train_data,train_lbl=[],[]
pth=["/content/drive/My Drive/AVEC_trans/metadata.csv"]
for p in pth:
    with open(p,"rt") as f:
        l_data=csv.reader(f)
        for r in l_data:
          if len(r):
            if r[0] in target and ('training' in r[0]):
#               print(r[0],r[3])
#               print(embedding_matrix[train_target.index(r[0])])
              train_data.append(f_data[target.index(r[0])])
              train_lbl.append(int(r[3])) #change  this t0 2 for DAIC

In [10]:
print(train_data[1])
print(train_lbl)
train_data=np.asarray(train_data,dtype=np.float32)
train_lbl=np.asarray(train_lbl,dtype=np.float32)
train_lbl=np.reshape(train_lbl,(-1,1))

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 3.44398499e+01  3.28299999e-02  3.37319412e+01 ...  3.86667013e-01
   4.24447000e-01 -4.39475288e+01]
 [ 3.50287285e+01  5.78010008e-02  3.38134193e+01 ...  2.48750001e-01
   3.06775004e-01 -4.35704918e+01]
 [ 3.47433395e+01  1.10545002e-01  3.07964802e+01 ...  6.63999975e-01
   1.10872197e+00 -5.00953102e+01]]
[4, 0, 6, 7, 4, 22, 15, 4, 21, 2, 7, 1, 2, 6, 3, 13, 5, 1, 5, 10, 2, 4, 4, 1, 12, 18, 5, 12, 10, 15, 11, 1, 7, 15, 23, 20, 5, 14, 10, 11, 18, 10, 10, 7, 7, 13, 4, 0, 20, 0, 0, 0, 19, 7, 0, 0, 13, 5, 12, 16, 1, 2, 10, 7, 15, 8, 11, 2, 14, 9, 9, 1, 7, 5, 5, 7, 7, 0, 0, 17, 2, 3, 10, 12, 0, 10, 16, 3, 7, 10, 3, 3, 10, 12, 3, 20, 5, 0, 1, 3, 1, 2, 0, 2, 1, 19, 1, 7,

In [11]:
print(train_data.shape)
print(train_lbl.shape)

(163, 1410, 88)
(163, 1)


In [12]:
test_data,test_lbl=[],[]


with open("/content/drive/My Drive/AVEC_trans/metadata.csv","rt") as f:
  l_data=csv.reader(f)
  for r in l_data:
    if len(r):
      if r[0] in target and('development' in r[0]):
        print(r[0],r[3])
        test_data.append(f_data[target.index(r[0])])
        test_lbl.append(int(r[3]))   #change  this t0 2 for DAIC
test_data=np.asarray(test_data,dtype=np.float32)
test_lbl=np.asarray(test_lbl,dtype=np.float32)
test_lbl=np.reshape(test_lbl,(-1,1)) 

development_01 2
development_02 3
development_03 0
development_04 8
development_05 11
development_06 20
development_07 8
development_08 5
development_09 7
development_10 9
development_11 11
development_12 16
development_13 11
development_14 12
development_15 9
development_16 9
development_17 2
development_18 16
development_19 0
development_20 17
development_21 2
development_22 9
development_23 11
development_24 0
development_25 12
development_26 3
development_27 0
development_28 6
development_29 2
development_30 10
development_31 8
development_32 0
development_33 18
development_34 6
development_35 18
development_36 4
development_37 1
development_38 1
development_39 2
development_40 4
development_41 0
development_42 6
development_43 7
development_44 1
development_45 9
development_46 4
development_47 12
development_48 3
development_49 1
development_50 4
development_51 0
development_52 3
development_53 0
development_54 7
development_55 19
development_56 0


In [13]:
print(test_data.shape)
print(test_lbl.shape)
print(test_lbl)
print(test_data[1])

(56, 1410, 88)
(56, 1)
[[ 2.]
 [ 3.]
 [ 0.]
 [ 8.]
 [11.]
 [20.]
 [ 8.]
 [ 5.]
 [ 7.]
 [ 9.]
 [11.]
 [16.]
 [11.]
 [12.]
 [ 9.]
 [ 9.]
 [ 2.]
 [16.]
 [ 0.]
 [17.]
 [ 2.]
 [ 9.]
 [11.]
 [ 0.]
 [12.]
 [ 3.]
 [ 0.]
 [ 6.]
 [ 2.]
 [10.]
 [ 8.]
 [ 0.]
 [18.]
 [ 6.]
 [18.]
 [ 4.]
 [ 1.]
 [ 1.]
 [ 2.]
 [ 4.]
 [ 0.]
 [ 6.]
 [ 7.]
 [ 1.]
 [ 9.]
 [ 4.]
 [12.]
 [ 3.]
 [ 1.]
 [ 4.]
 [ 0.]
 [ 3.]
 [ 0.]
 [ 7.]
 [19.]
 [ 0.]]
[[  0.         0.         0.       ...   0.         0.         0.      ]
 [  0.         0.         0.       ...   0.         0.         0.      ]
 [  0.         0.         0.       ...   0.         0.         0.      ]
 ...
 [ 33.07256    0.172561  27.55639  ...   0.435714   0.319189 -44.55476 ]
 [ 31.27195    0.228741  24.49269  ...   0.668      0.441198 -44.61617 ]
 [ 25.54603    0.081288  24.22628  ...   0.606      0.880968 -32.08368 ]]


In [0]:
test_data=np.reshape(test_data,[test_data.shape[0],test_data.shape[1],test_data.shape[2],1])
train_data=np.reshape(train_data,[train_data.shape[0],train_data.shape[1],train_data.shape[2],1])


In [15]:
print(test_data.shape)
print(train_data.shape)

(56, 1410, 88, 1)
(163, 1410, 88, 1)


In [0]:
def network(x,dp):
    conv1=tf.layers.conv2d(x,kernel_size=3,strides=2,filters = 32, padding="same")
   
    conv1=tf.nn.relu(conv1)
    conv1=tf.layers.batch_normalization(conv1)
    conv1=tf.nn.dropout(conv1,keep_prob=dp)
    print(conv1.shape)
    conv2 = tf.layers.conv2d(conv1,kernel_size=3,strides=2,filters = 64, padding="same")
    
    conv2=tf.nn.relu(conv2)
    conv2=tf.layers.batch_normalization(conv2)
    conv2=tf.nn.dropout(conv2,keep_prob=dp)
    print(conv2.shape)
    conv3 = tf.layers.conv2d(conv2,kernel_size=3,strides=2,filters = 128, padding="same")
    
    conv3=tf.nn.relu(conv3)
    conv3=tf.layers.batch_normalization(conv3)
    conv3=tf.nn.dropout(conv3,keep_prob=dp)
    print(conv3.shape)
    
    conv4 = tf.layers.conv2d(conv3,kernel_size=3,strides=2,filters = 128, padding="same")
    
    conv4=tf.nn.relu(conv4)
    conv4=tf.layers.batch_normalization(conv4)
    conv4=tf.nn.dropout(conv4,keep_prob=dp)
    print(conv4.shape)
    
    conv5 = tf.layers.conv2d(conv4,kernel_size=3,strides=2,filters = 128, padding="same")
    conv5=tf.nn.relu(conv5)
    conv5=tf.layers.batch_normalization(conv5)
    conv5=tf.nn.dropout(conv5,keep_prob=dp)
    print('conv5',conv5.shape)
    
    conv6 = tf.layers.conv2d(conv5,kernel_size=3,strides=2,filters = 512, padding="same")
    conv6=tf.nn.relu(conv6)
    conv6=tf.layers.batch_normalization(conv6)
    conv6=tf.nn.dropout(conv6,keep_prob=dp)
    print(conv6.shape)
    fc=tf.reshape(conv6,[-1,23*2*512])
    fc=tf.layers.batch_normalization(tf.nn.relu(tf.layers.dense(fc,1024)))
    fc=tf.nn.dropout(fc,keep_prob=dp)
    fc=tf.layers.batch_normalization(tf.nn.relu(tf.layers.dense(fc,512)))
    fc=tf.nn.dropout(fc,keep_prob=dp)
    fc=tf.layers.batch_normalization(tf.nn.relu(tf.layers.dense(fc,128)))
               
    logit= tf.layers.dense(fc,1)
    print(logit.shape)
    prob=tf.nn.sigmoid(logit)
    
    return logit,prob

In [0]:
# tf.reset_default_graph()
# batch_size=32
# beta=0.01
# x=tf.placeholder(tf.float32,[None,emb_dim,88,1])
# y=tf.placeholder(tf.float32,[None,1])
# drpouts= tf.placeholder(tf.float32)
# logit,prob=network(x,drpouts)
# # variables_to_train=tf.trainable_variables()
# # regularization_cost = tf.reduce_sum([ tf.nn.l2_loss(v) for v in variables_to_train if "bias" not in v.name ])
# # cost=tf.reduce_mean(tf.reduce_mean(tf.losses.mean_squared_error(labels=y,predictions=logit)) + beta*regularization_cost)
# cost=tf.reduce_mean(tf.losses.mean_squared_error(labels=y,predictions=logit))
# with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
#   optimizer=tf.train.AdamOptimizer(0.001).minimize(cost)
# # correct_prediction = tf.equal(tf.argmax(logit,1), tf.argmax(y, 1))
# # accuracay = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     for ep in range(50):   
#             sm=[]
#             train_data,train_lbl=shuffle(train_data,train_lbl)
#             for  j in range(train_data.shape[0]//batch_size):
#                     ls=train_data[j*batch_size:(j*batch_size+batch_size)]
#                     otp=train_lbl[j*batch_size:(j*batch_size+batch_size)]
                    
# #                     correct_prediction = tf.equal(tf.argmax(logit,1), tf.argmax(y, 1))
# #                     accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
                    
#                     _,loss=sess.run([optimizer,cost], feed_dict={x:ls,y:otp,drpouts:0.75})
#                     sm.append(loss)
#             print(sm)        
#             print("epoch: %d      train_loss: %f "%(ep,sum(sm)/len(sm)))
#             valid_loss= sess.run(cost, feed_dict={x:test_data,y :test_lbl,drpouts:1.0})
#             print("epoch: %d      test_loss: %f "%(ep,valid_loss))

In [0]:
test_data=np.reshape(test_data,[test_data.shape[0],test_data.shape[1],test_data.shape[2]])
train_data=np.reshape(train_data,[train_data.shape[0],train_data.shape[1],train_data.shape[2]])

In [0]:

# bidirectional lstms
def network2(xx,dp):
  
  with tf.variable_scope('GEN'):
#     print("kp=",kp[0],kp[1])
    lstms_fw = [tf.nn.rnn_cell.LSTMCell(size,use_peepholes=True) for size in [200,200]]
    lstms_bw = [tf.nn.rnn_cell.LSTMCell(size,use_peepholes=True) for size in [200,200]]
#     print('fw',lstms_fw)
    drops_fw = [tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=1.0,variational_recurrent=False,dtype=tf.float32) for lstm in lstms_fw]
    drops_bw= [tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=1.0,variational_recurrent=False,dtype=tf.float32) for lstm in lstms_bw]

#      (output_fw, output_bw),final_state =tf.nn.bidirectional_dynamic_rnn(drops_fw,drops_bw,xx)
    lstm_output,output_fw, output_bw =tf.contrib.rnn.stack_bidirectional_dynamic_rnn(cells_fw=drops_fw,cells_bw=drops_bw,inputs=xx,dtype=tf.float32)
    print('lstm',lstm_output.shape)
#     print(output_fw)
    fs=lstm_output[:, -1]
#     fs=tf.reduce_sum(lstm_output,1)
#     context_vec,attention_wts = Attention(512).call(lstm_output,lstm_output[:,-1])
#     logits=tf.nn.dropout(fs,keep_prob=dp)
#     logits=tf.layers.batch_normalization(tf.nn.relu(tf.layers.dense(logits,500)))
    logits=tf.layers.batch_normalization(tf.nn.relu(tf.layers.dense(fs,500)))
#     logits= tf.nn.relu(tf.layers.dense(context_vec,112))
    logits=tf.nn.dropout(logits,keep_prob=dp)
    logits=tf.layers.batch_normalization(tf.nn.relu(tf.layers.dense(logits,100)))
    logits=tf.nn.dropout(logits,keep_prob=dp)
    logits=tf.layers.batch_normalization(tf.nn.relu(tf.layers.dense(logits,60)))
    logits= tf.layers.dense(logits,1)
    
    return logits
  

In [0]:
path = "/content/drive/My Drive/saved_model_f_ege/model.ckpt"
mm=34

In [0]:

for _ in range(1):
  tf.reset_default_graph()
  batch_size=10
  x=tf.placeholder(tf.float32,[None,emb_dim,88])
  y=tf.placeholder(tf.float32,[None,1])
  train_phase = tf.placeholder(tf.bool, name="is_training")
  drpouts= tf.placeholder(tf.float32)
  logit=network2(x,drpouts)
  cost=tf.reduce_mean(tf.losses.mean_squared_error(labels=y,predictions=logit))
  with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
    optimizer=tf.train.AdamOptimizer(0.0001).minimize(cost)
  # correct_prediction = tf.equal(tf.argmax(logit,1), tf.argmax(y, 1))
  # accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  saver = tf.train.Saver()
  with tf.Session() as sess:
      try:
        saver.restore(sess,path)
        print('restored')
        los_test= sess.run(cost, feed_dict={x:test_data,y :test_lbl,drpouts:1.0,train_phase:False})
  #       los_val= sess.run(loss,feed_dict={X:val_data,Y:val_lbl,keep_prob:[1.0,1.0],drpouts:1.0,batch_size:len(val_data)})
        print("TEST LOSS FROM RESTORED MODEL AVEC : ", los_test)
  #       print("VAL ON DAIC : ", los_val)
        mm = los_test
      except:
        print('No pre-saved weights found')  
        sess.run(tf.global_variables_initializer())
  #     sess.run(tf.global_variables_initializer())
      for ep in range(25):   
              sm=[]
              train_data1,train_lbl1=shuffle(train_data,train_lbl)
              for  j in range(train_data.shape[0]//batch_size):
                      ls=train_data1[j*batch_size:(j*batch_size+batch_size)]
                      otp=train_lbl1[j*batch_size:(j*batch_size+batch_size)]

  #                     correct_prediction = tf.equal(tf.argmax(logit,1), tf.argmax(y, 1))
  #                     accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

                      _,loss=sess.run([optimizer,cost], feed_dict={x:ls,y:otp,drpouts:1.0,train_phase:True})
                      sm.append(loss)
              print(sm)        
              print("epoch: %d      train_loss: %f "%(ep,sum(sm)/len(sm)))
              c_loss=sum(sm)/len(sm)
              valid_loss= sess.run(cost, feed_dict={x:test_data,y :test_lbl,drpouts:1.0,train_phase:False})
              print("epoch: %d      test_loss: %f "%(ep,valid_loss))
              if (abs(c_loss-valid_loss)<3) and (valid_loss<mm):
                save_path = saver.save(sess, path)
                mm=valid_loss
                print('saved')

lstm (?, 1410, 400)
restored
TEST LOSS FROM RESTORED MODEL AVEC :  30.489824
[46.40744, 25.053013, 25.620548, 23.600113, 29.27074, 18.344196, 31.052088, 55.396507, 26.486496, 19.875814, 19.229115, 14.999415, 34.11844, 34.27511, 34.985554, 30.23523]
epoch: 0      train_loss: 29.309364 
epoch: 0      test_loss: 30.524883 
[29.221203, 27.977045, 43.26657, 23.29655, 35.49923, 13.689288, 20.27438, 30.458155, 19.800861, 18.028671, 24.066788, 36.250492, 32.7524, 52.070305, 21.099588, 23.581959]
epoch: 1      train_loss: 28.208343 
epoch: 1      test_loss: 30.714874 
[26.391598, 19.82381, 22.030901, 13.932299, 22.959885, 26.78741, 21.890148, 16.513409, 23.888134, 26.296392, 42.790226, 47.49276, 47.87064, 39.03817, 20.8877, 23.730846]
epoch: 2      train_loss: 27.645270 
epoch: 2      test_loss: 30.889713 
[15.811325, 35.181923, 45.977337, 24.524601, 51.647346, 33.455585, 23.020084, 23.787374, 19.920183, 17.298637, 14.286761, 58.19686, 4.9870677, 17.06947, 21.020409, 37.22033]
epoch: 3      tra

In [0]:
#SUM OF TIME STEPS

In [0]:
# tf.reset_default_graph()
# batch_size=20
# x=tf.placeholder(tf.float32,[None,emb_dim,88])
# y=tf.placeholder(tf.float32,[None,1])
# train_phase = tf.placeholder(tf.bool, name="is_training")
# drpouts= tf.placeholder(tf.float32)
# logit=network2(x,drpouts)
# cost=tf.reduce_mean(tf.losses.mean_squared_error(labels=y,predictions=logit))
# with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
#   optimizer=tf.train.AdamOptimizer(0.001).minimize(cost)
# # correct_prediction = tf.equal(tf.argmax(logit,1), tf.argmax(y, 1))
# # accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     for ep in range(100):   
#             sm=[]
#             train_data1,train_lbl1=shuffle(train_data,train_lbl)
#             for  j in range(train_data.shape[0]//batch_size):
#                     ls=train_data1[j*batch_size:(j*batch_size+batch_size)]
#                     otp=train_lbl1[j*batch_size:(j*batch_size+batch_size)]
                    
# #                     correct_prediction = tf.equal(tf.argmax(logit,1), tf.argmax(y, 1))
# #                     accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
                    
#                     _,loss=sess.run([optimizer,cost], feed_dict={x:ls,y:otp,drpouts:1.0,train_phase:True})
#                     sm.append(loss)
#             print(sm)        
#             print("epoch: %d      train_loss: %f "%(ep,sum(sm)/len(sm)))
#             valid_loss= sess.run(cost, feed_dict={x:test_data,y :test_lbl,drpouts:1.0,train_phase:False})
#             print("epoch: %d      test_loss: %f "%(ep,valid_loss))